In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from brian2 import *
import matplotlib.pyplot as plt
from scipy.stats import norm

import trials
import plotter

In [3]:
def compute_headings(h, N=8, loc=0, scale=0.8, vmin=5, vmax=100):
    rv = norm(loc=loc, scale=scale)
    x = np.linspace(rv.ppf(0.01), rv.ppf(0.99), N, endpoint=True)
    pdf = rv.pdf(x)

    # Split the [-pi,pi] interval into bins
    bins = numpy.linspace(-np.pi, np.pi, N+1)

    # -1 required since the returned bins are 1-indexed
    digitized = numpy.digitize(h, bins) - 1
    
    headings = np.zeros((len(h), N))

    for t in range(T_outbound):
        # Shift angle
        angle_shift = N//2 + digitized[t]
        headings[t,:] = np.roll(pdf, angle_shift)
    
    # Normalize between 5-100 Hz, the headings represents rate
    if vmin > 0 and vmin > 0 and vmax > vmin:
        headings = ((headings - headings.min()) * (vmax - vmin)) / (headings.max() - headings.min()) + vmin
    
    return headings, digitized    

    
T_outbound = 1500
h, v = trials.generate_route(T=T_outbound, vary_speed=True)
headings, digitized = compute_headings(h, N=N_TL2//2, vmin=5, vmax=30)
headings = np.tile(headings, 2)

In [6]:
start_scope()
time_step = 10
stimulus = TimedArray(headings*Hz, dt=1.*time_step*ms)
P = PoissonGroup(16, rates='stimulus(t,i)')
spike_monitor = SpikeMonitor(P)

In [7]:
run(T_outbound*time_step*ms)

In [88]:
start_scope()

# Constants
N_TL2 = 16

# Neuron specification
El = -52 * mV # resting potential (mV)
Vt = -45 * mV # spike threshold (mV)
Vr = El       # reset potential (mV)

Cm = 0.002 * ufarad # membrane capacitance (uF)
Rm = 10 * Mohm      # membrane resistance (MOhm)
taum = Cm * Rm      # = 20ms membrane time constant (ms) 

#ge = 0 * mV # excitatory synaptic conductance ()
#gi = 0 * mV # inhibitory synaptic conductance ()
taue = 5 * ms # excitatory synaptic time constant (ms)
taui = 10 * ms # inhibitory synaptic time constant (ms)


eqs = '''
      dVm/dt  = ((El - Vm) + I_syn) / taum : volt
      I_syn = ge + gi : volt
      dge/dt = -ge/taue : volt
      dgi/dt = -gi/taui : volt
      '''
threshold_eqs = 'Vm >= Vt'
reset_eqs = 'Vm = Vr'

G_TL2 = NeuronGroup(N=N_TL2, model=eqs, threshold=threshold_eqs, reset=reset_eqs, method='euler')
G_TL2.Vm = Vr
G_TL2.ge = 0 * mV
G_TL2.gi = 0 * mV

In [89]:
def add_monitors(NG, variables_to_record=['Vm', 'ge', 'gi']):
    state_monitor = StateMonitor(NG, variables_to_record, record=True)
    spike_monitor = SpikeMonitor(NG)
    return state_monitor, spike_monitor

STM_TL2, SPM_TL2 = add_monitors(G_TL2)

In [90]:
we = (60 * 0.27 / 10) * mV
wi = (20 * 4.5 / 10) * mV

synapses_model = '''w : 1'''
synapses_eqs_ex = '''ge += we * w'''
synapses_eqs_in = '''gi += wi * w'''

In [91]:
#stimulus = TimedArray(np.tile([100., 0.], 2)*Hz, dt=100.*ms)
stimulus = TimedArray(100*h*Hz, dt=1500.*ms)

P = PoissonGroup(N_TL2, rates='stimulus(t)')
S = Synapses(P, G_TL2, model=synapses_model, on_pre=synapses_eqs_in)
S.connect(j='i')
S.w = 1

In [92]:
run(1500*ms)

In [52]:
len(spike_monitor.t/ms)

3944

In [1]:
figure(figsize=(10,5))
bins = numpy.linspace(-np.pi, np.pi, 8+1)
plt.plot(h)
for b in bins:
    plt.axhline(b, ls='dashed', color='r', alpha=0.5)
ylabel('angle[-pi, pi]')
xlabel('steps')
#plt.savefig('path.pdf')
plt.show()

figure(figsize=(10,5))
plot(spike_monitor.t/ms, spike_monitor.i, '.k', markersize=6)
plot(np.array(range(0, T_outbound*time_step, time_step)), np.argmax(headings,axis=1),'xr', markersize=3,   alpha=0.2)
plot(np.array(range(0, T_outbound*time_step, time_step)), 8+np.argmax(headings,axis=1),'xr', markersize=3,   alpha=0.2)
ylabel('neuron index')
xlabel('ms')
#plt.savefig('spikes.pdf')
plt.show()

NameError: name 'figure' is not defined